In [1]:
import pandas as pd

# Función para cargar los archivos
def cargar_archivos(padron_file, rendicion_file):
    padron_df = pd.read_excel(padron_file)
    rendicion_df = pd.read_excel(rendicion_file, skiprows=7)
    return padron_df, rendicion_df

def contar_coincidencias(nombre_rendicion, nombre_padron):
    """
    Función para contar coincidencias entre el nombre de rendición y el nombre del padrón.
    Devuelve True si hay al menos dos coincidencias, False en caso contrario.
    """
    # Separar los nombres en palabras
    palabras_rendicion = set(nombre_rendicion.split())
    palabras_padron = set(nombre_padron.split())
    
    # Contar coincidencias
    coincidencias = palabras_rendicion.intersection(palabras_padron)
    
    # Debug: Mostrar las palabras y las coincidencias
    print(f"Palabras en rendición: {palabras_rendicion}")
    print(f"Palabras en padrón: {palabras_padron}")
    print(f"Coincidencias: {coincidencias} (Total: {len(coincidencias)})")
    
    # Retornar True si hay al menos 2 coincidencias
    return len(coincidencias) >= 2

# Función para procesar los DataFrames y realizar la comparación
def procesar_comparacion(padron_df, rendicion_df):
    # Unir "Apellido" y "Nombre" en el padrón para facilitar la comparación
    padron_df['Apellido y Nombre'] = padron_df['Apellido'] + ' ' + padron_df['Nombre']
    
    # Crear una nueva columna para indicar si la fila es correcta o tiene errores
    resultado = []
    
    for i, fila_rendicion in rendicion_df.iterrows():
        # Obtener la credencial y el nombre completo de la rendición
        credencial_rendicion = fila_rendicion['Cred']
        nombre_rendicion = fila_rendicion['Apellido y Nombres']
        
        # Buscar la credencial en el padrón
        fila_padron = padron_df[padron_df['Credencial'] == credencial_rendicion]
        
        # Verificar si la credencial existe y si el nombre coincide
        if not fila_padron.empty:
            nombre_padron = fila_padron.iloc[0]['Apellido y Nombre']
            
            # Debug: mostrar la comparación
            print(f"Comparando: {nombre_padron} (padrón) == {nombre_rendicion} (rendición)")
            
            if contar_coincidencias(nombre_rendicion, nombre_padron):
                resultado.append("Correcto")
            else:
                print(f"Error en nombre: {nombre_padron} != {nombre_rendicion}")  # Debug
                resultado.append("Error en nombre")
        else:
            print(f"Error en credencial: {credencial_rendicion} no se encontró en el padrón")  # Debug
            resultado.append("Error en credencial")
    
    # Agregar la columna de resultado al DataFrame de rendición
    rendicion_df['Resultado'] = resultado
    
    # Guardar el resultado en un nuevo archivo Excel
    rendicion_df.to_excel('RENDICON VENTAS COD608 P05-24 (CONTROLADO).xlsx', index=False)
    
    return rendicion_df

# Función principal que encapsula todo el proceso
def ejecutar_proceso(padron_file, rendicion_file):
    # Cargar los archivos
    padron_df, rendicion_df = cargar_archivos(padron_file, rendicion_file)
    
    # Procesar los DataFrames y hacer la comparación
    rendicion_controlada_df = procesar_comparacion(padron_df, rendicion_df)
    
    print("Proceso completado. Archivo guardado como 'RENDICON VENTAS COD608 P05-24 (CONTROLADO).xlsx'.")
    print(rendicion_controlada_df.head())  # Mostrar una vista previa del archivo resultante


In [2]:
# Llamar a la función principal pasando los archivos
padron_file = 'PADRON ACTIVIDAD 21-05-24.xlsx'
rendicion_file = 'RENDICON VENTAS COD608 P05-24 (1).xlsx'

# Ejecutar el proceso
ejecutar_proceso(padron_file, rendicion_file)

Comparando: FERNANDEZ MATIAS GREGORIO (padrón) == MATIAS FERNANDEZ (rendición)
Palabras en rendición: {'MATIAS', 'FERNANDEZ'}
Palabras en padrón: {'GREGORIO', 'MATIAS', 'FERNANDEZ'}
Coincidencias: {'MATIAS', 'FERNANDEZ'} (Total: 2)
Comparando: VILLAGRA JESICA LOURDES (padrón) == JESICA VILLAGRA (rendición)
Palabras en rendición: {'VILLAGRA', 'JESICA'}
Palabras en padrón: {'VILLAGRA', 'JESICA', 'LOURDES'}
Coincidencias: {'VILLAGRA', 'JESICA'} (Total: 2)
Comparando: AYZAGUER FRANCO DA SILVA ELIANA MARIEL (padrón) == ELIANA AYZAGUER (rendición)
Palabras en rendición: {'ELIANA', 'AYZAGUER'}
Palabras en padrón: {'ELIANA', 'MARIEL', 'SILVA', 'AYZAGUER', 'DA', 'FRANCO'}
Coincidencias: {'ELIANA', 'AYZAGUER'} (Total: 2)
Comparando: BOGADO VALERIA JESICA (padrón) == VALERIA BOGADO (rendición)
Palabras en rendición: {'VALERIA', 'BOGADO'}
Palabras en padrón: {'VALERIA', 'BOGADO', 'JESICA'}
Coincidencias: {'VALERIA', 'BOGADO'} (Total: 2)
Comparando: HUENTEMILLA DARIO OSVALDO (padrón) == DARIO HUENT

c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


# Obtener facturas pendientes de descuento

In [7]:
import pandas as pd
from openpyxl import load_workbook

def obtener_facturas_pendientes(nombre_archivo, nombre_hoja):
    # Leer la hoja específica del archivo Excel
    df = pd.read_excel(nombre_archivo, sheet_name=nombre_hoja)

    # Asegurarse de que las columnas relevantes están presentes
    required_columns = ['CUOTA N°', 'CTA. COMPU.', 'FACT.', 'CUOTA MES']
    if not all(col in df.columns for col in required_columns):
        raise ValueError("Faltan columnas necesarias en el archivo Excel.")
    
    # Convertir las columnas relevantes a tipo numérico
    df['CUOTA N°'] = pd.to_numeric(df['CUOTA N°'], errors='coerce')
    df['CTA. COMPU.'] = pd.to_numeric(df['CTA. COMPU.'], errors='coerce')

    # Filtrar filas con valores NaN en las columnas clave
    df = df.dropna(subset=['CUOTA N°', 'CTA. COMPU.'])

    # Agrupar por 'CUOTA MES' y 'FACT.' y conservar solo la fila con el mayor 'CUOTA N°'
    facturas_filtradas = df.loc[df.groupby(['CUOTA MES', 'FACT.'])['CUOTA N°']]

    # Filtrar filas según la condición de CTA. COMPU. y CUOTA N°
    facturas_finales = facturas_filtradas[
        facturas_filtradas['CTA. COMPU.'] <= facturas_filtradas['CUOTA N°']
    ]

    # Ajustar formato de las columnas, si es necesario
    facturas_finales['CUOTA MES'] = facturas_finales['CUOTA MES'].astype(float).round(2)
    facturas_finales['FCHA OPER.'] = pd.to_datetime(facturas_finales['FCHA OPER.'], errors='coerce').dt.strftime('%d/%m/%Y')

    # Devolver el DataFrame con las facturas filtradas
    return facturas_finales

In [8]:
# Ejemplo de uso
nombre_archivo = 'C1 COD 608 DESDE C 07-2012.xlsx'
nombre_hoja = 'C 11-24'  # Nombre de la hoja

try:
    facturas_pendientes = obtener_facturas_pendientes(nombre_archivo, nombre_hoja)

    # Guardar el resultado en un nuevo archivo Excel
    facturas_pendientes.to_excel('facturas_pendientes.xlsx', index=False)
    print("El archivo 'facturas_pendientes.xlsx' ha sido creado con éxito.")
except Exception as e:
    print(f"Ocurrió un error: {e}")

Ocurrió un error: Cannot index with multidimensional key


c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [17]:
# Definir la función para obtener facturas pendientes
def obtener_facturas_pendientes2(nombre_archivo, nombre_hoja):
    # Cargar el archivo Excel
    df = pd.read_excel(nombre_archivo, sheet_name=nombre_hoja)

    # Filtrar las filas donde OBSERV es 'C 11-24' y CTA. COMPU >= CUOTA N°
    df_filtrado = df[(df['OBSERV'] == nombre_hoja) & (df['CTA. COMPU.'] >= df['CUOTA N°'])]

    # Convertir las columnas a cadenas antes de concatenar
    df_filtrado['CUOTA MES'] = df_filtrado['CUOTA MES'].astype(str)
    df_filtrado['FACT.'] = df_filtrado['FACT.'].astype(str)
    df_filtrado['DETALLE DE LA COMPRA'] = df_filtrado['DETALLE DE LA COMPRA'].astype(str)

    # Crear un índice con CUOTA MES + FACT + DETALLE DE LA COMPRA
    df_filtrado.set_index(df_filtrado['CUOTA MES'] + ' ' + df_filtrado['FACT.'] + ' ' + df_filtrado['DETALLE DE LA COMPRA'], inplace=True)

    # Agregar una columna 'Estado' que indique "Cancelado" si CTA. COMPU. es igual a CUOTA N°
    df_filtrado['Estado'] = df_filtrado.apply(lambda x: 'Cancelado' if x['CTA. COMPU.'] == x['CUOTA N°'] else '', axis=1)


    df_filtrado['CUOTA MES'] = df_filtrado['CUOTA MES'].astype(float).round(2)
    df_filtrado['FCHA OPER.'] = pd.to_datetime(df_filtrado['FCHA OPER.'], errors='coerce').dt.strftime('%d/%m/%Y')
    df_filtrado['FACT.']= pd.to_numeric(df_filtrado['FACT.'], errors='coerce')
    # Convertir las columnas relevantes a tipo numérico
    df_filtrado['CUOTA N°'] = pd.to_numeric(df_filtrado['CUOTA N°'], errors='coerce')
    df_filtrado['CTA. COMPU.'] = pd.to_numeric(df_filtrado['CTA. COMPU.'], errors='coerce')

    # Retornar el DataFrame filtrado
    return df_filtrado

In [18]:

# Ejemplo de uso
nombre_archivo = 'C1 COD 608 DESDE C 07-2012.xlsx'
nombre_hoja = 'C 10-24'  # Nombre de la hoja

try:
    facturas_pendientes = obtener_facturas_pendientes2(nombre_archivo, nombre_hoja)

    # Guardar el resultado en un nuevo archivo Excel
    facturas_pendientes.to_excel('facturas_pendientes.xlsx', index=False)
    print("El archivo 'facturas_pendientes.xlsx' ha sido creado con éxito.")
except Exception as e:
    print(f"Ocurrió un error: {e}")

El archivo 'facturas_pendientes.xlsx' ha sido creado con éxito.


c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\administrator\AppData\Local\Temp\ipykernel_15472\2662049815.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['CUOTA MES'] = df_filtrado['CUOTA MES'].astype(str)
C:\Users\administrator\AppData\Local\Temp\ipykernel_15472\2662049815.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['FACT.'] = df_filtrado['FACT.'].astype